## 2. Lists and Dictionaries

### 18 Know How to Construct Key-Dependent Default Values with `__missing__`

In [1]:
import logging

In [2]:
pictures = {}
path = 'profile_1234.png'

with open(path, 'wb') as f:
    f.write(b'image data here 1234')

if (handle := pictures.get(path)) is None:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()

print(pictures)
print(image_data)

{'profile_1234.png': <_io.BufferedRandom name='profile_1234.png'>}
b'image data here 1234'


In [3]:
pictures = {}
path = 'profile_9991.png'

with open(path, 'wb') as f:
    f.write(b'image data here 9991')

if path in pictures:
    handle = pictures[path]
else:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()

print(pictures)
print(image_data)

{'profile_9991.png': <_io.BufferedRandom name='profile_9991.png'>}
b'image data here 9991'


In [4]:
pictures = {}
path = 'profile_9922.png'

with open(path, 'wb') as f:
    f.write(b'image data here 9991')

try:
    handle = pictures[path]
except KeyError:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()

print(pictures)
print(image_data)

{'profile_9922.png': <_io.BufferedRandom name='profile_9922.png'>}
b'image data here 9991'


In [5]:
pictures = {}
path = 'profile_9239.png'

with open(path, 'wb') as f:
    f.write(b'image data here 9239')

try:
    handle = pictures.setdefault(path, open(path, 'a+b'))
except OSError:
    print(f'Failed to open path {path}')
    raise
else:
    handle.seek(0)
    image_data = handle.read()

print(pictures)
print(image_data)

# 딕셔너리에 이미 path와 그 파일 핸들이 있어도
# open()이 항상 호출된다
# 따라서 이 경우에는 setdefault 를 쓰지 않는 게 좋겠다.

{'profile_9239.png': <_io.BufferedRandom name='profile_9239.png'>}
b'image data here 9239'


In [6]:
path = 'profile_4555.png'

with open(path, 'wb') as f:
    f.write(b'image data here 4555')

def open_picture(profile_path):
    try:
        return open(profile_path, 'a+b')
    except OSError:
        print(f'Failed to open path {profile_path}')
        raise

try:
    from collections import defaultdict
    
    pictures = defaultdict(open_picture)
    handle = pictures[path]
    handle.seek(0)
    image_data = handle.read()
except:
    logging.exception('Expected')
else:
    assert False

#print(pictures)
#print(image_data)

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-732624fe9cba>", line 17, in <module>
    handle = pictures[path]
TypeError: open_picture() missing 1 required positional argument: 'profile_path'


In [7]:
path = 'profile_9090.png'

with open(path, 'wb') as f:
    f.write(b'image data here 9090')

def open_picture(profile_path):
    try:
        return open(profile_path, 'a+b')
    except OSError:
        print(f'Failed to open path {profile_path}')
        raise

class Pictures(dict):
    def __missing__(self, key):
        value = open_picture(key)
        self[key] = value
        return value

pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()

print(pictures)
print(image_data)

{'profile_9090.png': <_io.BufferedRandom name='profile_9090.png'>}
b'image data here 9090'


> - 디폴트 값을 만드는 계산 비용이 높거나 만드는 과정에서 예외가 발생할 수 있는 상황에서는 `dict`의 `setdefault` 메서드를 사용하지 말라.
> - `defaultdict`에 전달되는 함수는 인자를 받지 않는다. 따라서 접근에 사용한 키 값에 맞는 디폴트 값을 생성하는 것은 불가능하다.
> - 디폴트 키를 만들 때 어떤 키를 사용했는지 반드시 알아야 하는 상황이라면 직접 `dict`의 하위 클래스와 `__missing__` 메서드를 정의하면 된다.